In [1]:
import bhnerf
from astropy import units
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
import ruamel.yaml as yaml
from tqdm.auto import tqdm
from bhnerf import constants as consts

data_path = Path('../data/synthetic_lightcurves')

import warnings
warnings.simplefilter("ignore")

# ALMA-like scans on Apr11, 2017 (time-averaged over a window)
t_frames = np.array([ 9.34056333,  9.35067   ,  9.36077667,  9.37088333,  9.38099   ,
                      9.39109667,  9.40120333,  9.41131   ,  9.42169667,  9.43405   ,
                      9.44415667,  9.45426333,  9.46437   ,  9.47447667,  9.48458333,
                      9.49546167,  9.56607333,  9.57618   ,  9.58628667,  9.59639333,
                      9.6065    ,  9.61660667,  9.62671333,  9.63682   ,  9.64889333,
                      9.65956   ,  9.66966667,  9.67977333,  9.68988   ,  9.69998667,
                      9.71191667,  9.91607333,  9.92618   ,  9.93881333,  9.94892   ,
                      9.95902667,  9.96913333,  9.97924   ,  9.98962667, 10.00071667,
                     10.01082333, 10.02093   , 10.03103667, 10.04114333, 10.05139   ,
                     10.06262   , 10.13274   , 10.14284667, 10.15295333, 10.16306   ,
                     10.17316667, 10.18327333, 10.19338   , 10.20348667, 10.21359333,
                     10.2237    , 10.23507   , 10.24644   , 10.28883833, 10.30975667,
                     10.31986333, 10.32997   , 10.34007667, 10.35018333, 10.36029   ,
                     10.37039667, 10.381415  , 10.39187333, 10.40198   , 10.41405333,
                     10.42472   , 10.43482667, 10.44493333, 10.45504   , 10.73603833,
                     10.74663667, 10.75674333, 10.76685   , 10.77695667, 10.78706333,
                     10.79808167, 10.80854   , 10.81977   , 10.83632167, 10.90067   ,
                     10.91077667, 10.92088333, 10.93211333, 10.94236   , 10.95246667,
                     10.96257333, 10.97345167, 10.98405   , 10.99415667, 11.00426333,
                     11.01437   , 11.02447667, 11.03584667,
                     11.10596667, 11.116565  , 11.12744333, 11.13755   , 11.14765667,
                     11.15776333, 11.16801   , 11.17924   , 11.19201333, 11.20324333,
                     11.21335   , 11.22345667, 11.23356333, 11.24367   , 11.25377667,
                     11.28678833, 11.45723   , 11.46733667, 11.47744333, 11.48881333,
                     11.50018333, 11.51092167, 11.52166   , 11.53176667, 11.54187333,
                     11.55198   , 11.56208667, 11.57219333, 11.5823    , 11.59240667]) * units.hr

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-95va40t2 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2023-04-06 17:04:03.840763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


# Generate 3D emissivities
---
Generate different emissivity structures for recovery:
 - Single hotspot
 - Three Hotspots
 - Hotpot + tube
 - Accretion disk

### Single Gaussian hotspot

In [15]:
fov_M = 40.0
hs_fwhm = 3
hs_std = hs_fwhm / 2.355
orbit_radius = 11.0
rot_angle = np.deg2rad(120) + np.pi/2
emission = bhnerf.emission.generate_hotspot_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    rot_angle=rot_angle,
    orbit_radius=orbit_radius,
    std=hs_std,
    r_isco=6.0,
    fov=(fov_M, 'GM/c^2'))
emission.name = 'Single Gaussian'
emission.attrs = {
    'fov_M': fov_M,
    'hs_fwhm': hs_fwhm,
    'orbit_radius': orbit_radius,
    'rot_angle': rot_angle
}

# Save ground truth emission 
directory = data_path.joinpath('single_gaussian')
directory.mkdir(parents=True, exist_ok=True)
emission.to_netcdf(directory.joinpath('emission_3d.nc'))

In [20]:
%matplotlib widget
bhnerf.visualization.ipyvolume_3d(emission, fov_M)

### Three Gaussian hotspots

In [10]:
fov_M = 40.0
hs_fwhms = np.array([3.4, 4.2, 4.6])
hs_stds = hs_fwhms / 2.355
orbit_radii = [6.5, 11.0, 13.0]
rot_angles = np.deg2rad(np.array([9.5, 72.3, 221.8]))

emission = 0
for hs_std, orbit_radius, rot_angle in zip(hs_stds, orbit_radii, rot_angles):
    emission +=  bhnerf.emission.generate_hotspot_xr(
        resolution=(64, 64, 64), 
        rot_axis=[0.0, 0.0, 1.0], 
        rot_angle=rot_angle,
        orbit_radius=orbit_radius,
        std=hs_std,
        r_isco=6.0,
        fov=(fov_M, 'GM/c^2'))
emission.name = 'Three Gaussians'
emission.attrs = {
    'fov_M': fov_M,
    'hs_fwhm': hs_fwhms,
    'orbit_radius': orbit_radii,
    'rot_angle': rot_angles
}

# Save ground truth emission 
directory = data_path.joinpath('3_gaussians')
directory.mkdir(parents=True, exist_ok=True)
emission.to_netcdf(directory.joinpath('emission_3d.nc'))

In [6]:
%matplotlib widget
bhnerf.visualization.ipyvolume_3d(emission, fov_M)

### Flux Tube

In [11]:
fov_M = 40.0
hs_std = 1.2
tube_std = 1.
tube_radius = 10.0
phi_start = np.deg2rad(190)
phi_end = np.deg2rad(270)

emission = bhnerf.emission.generate_tube_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    phi_start=phi_start,
    phi_end=phi_end,
    orbit_radius=tube_radius,
    std=tube_std,
    r_isco=6.0,
    fov=(fov_M, 'GM/c^2'))
emission.name = 'Flux Tube'
emission.attrs = {
    'fov_M': fov_M,
    'hs_std': hs_std,
    'tube_std':tube_std,
    'tube_radius':tube_radius
}

# Save ground truth emission 
directory = data_path.joinpath('flux_tube')
directory.mkdir(parents=True, exist_ok=True)
emission.to_netcdf(directory.joinpath('emission_3d.nc'))

In [80]:
%matplotlib widget
bhnerf.visualization.ipyvolume_3d(emission, fov_M)

### Accretion Disk (Gaussian Random Field)

In [16]:
grf_path = Path('../data/synthetic_lightcurves/accretion_grfs/grf.evo_140min_411M.seed_0.nc')
fov_M = 40.0
alpha = 0.5
diameter_M = 20   
H_r = 0            
std_z = 1

GM_c3 = consts.GM_c3(consts.sgra_mass).to('hr')
grf = xr.load_dataarray(grf_path).interp(t=(t_frames-t_frames[0]) / GM_c3.value)

gaussian = bhnerf.utils.gaussian_xr([grf.y.size, grf.x.size], [0,0], std=diameter_M/2.355, fov=(fov_M, 'M')).data
emission =  bhnerf.utils.expand_3d(np.exp(alpha*grf) * gaussian, fov_z=fov_M, H_r=H_r, std=std_z)
emission.coords.update(bhnerf.utils.linspace_xr(emission.shape[1:], -fov_M/2, fov_M/2))
emission.name = 'Accretion Disk'
emission.attrs = grf.attrs
emission.attrs.update({
    'grf_path': str(grf_path),
    'alpha': alpha,
    'diameter_M':diameter_M,
    'H_r': H_r, 
    'std_z': std_z
})

# Save ground truth emission 
emission.to_netcdf(grf_path.parent.joinpath('emission_3d_seed_{}.nc'.format(grf.seed)))

In [11]:
%matplotlib widget
bhnerf.visualization.ipyvolume_3d(emission, fov_M)

interactive(children=(IntSlider(value=0, description='t', max=127), Output()), _dom_classes=('widget-interact'…

# Generate lightcurves

In [61]:
def lc_average_fluxes(lightcurves, t_window_hr=100.0/60.0):
    radio_loops = lightcurves.where(lightcurves.t < lightcurves.t[0] + t_window_hr, drop=True)
    I = float(radio_loops.sel(stokes='I').mean('t'))
    P = float(np.sqrt(radio_loops.sel(stokes='Q')**2 + radio_loops.sel(stokes='U')**2).mean('t'))
    return I, P

def lc_polarized_std(lightcurves, t_window_hr=100.0/60.0):     
    radio_loops = lightcurves.where(lightcurves.t < lightcurves.t[0] + t_window_hr, drop=True)
    P_std = float((radio_loops.sel(stokes='Q') + 1j*radio_loops.sel(stokes='U')).std('t'))
    return P_std

def lc_lp_angle(lightcurves, t_window_hr=100.0/60.0):
    radio_loops = lightcurves.where(lightcurves.t < lightcurves.t[0] + t_window_hr, drop=True)
    chi = np.rad2deg(np.angle(radio_loops.sel(stokes='Q').mean('t') + 1j*radio_loops.sel(stokes='U').mean('t')) / 2.0)
    return chi

In [5]:
simulation_params = {
    'name': 'sim1',
    'desc': 'Image fluxes without background accretion',
    'model': {
        # Domain dimensions and black-hole params
        'spin': 0.0,
        'inclination': 12.0,
        'fov_M': 40.0, 
        'num_subrays': 10,
        
        # Magnetic field / polarization parameters
        'I_flux': 0.3,
        'Q_frac': 0.85,      # Fraction of linear polarization
        'b_consts': {
            'arad': 0,       # Radial component (in lab-frame) 
            'avert': 1,      # Vertical component (in lab-frame)
            'ator': 0,       # Toroidal component (in lab-frame)
        },
        # Keplerian velocity field
        'Omega_dir': 'cw',
        
        # Image plane 
        'num_alpha': 64,
        'num_beta': 64,
        't_start_obs': 9.3 
    }}

In [31]:
simulation_params = {
    'name': 'sim2',
    'desc': 'Image fluxes without background accretion',
    'flare_path': '../data/synthetic_lightcurves/3_gaussians/emission_3d_scale_0.1.nc',
    'model': {
        # Domain dimensions and black-hole params
        'spin': 0.0,
        'inclination': 64.0,
        'fov_M': 40.0, 
        
        # Magnetic field / polarization parameters
        'Q_frac': 0.5,      # Fraction of linear polarization
        'b_consts': {
            'arad': 0,       # Radial component (in lab-frame) 
            'avert': 1,      # Vertical component (in lab-frame)
            'ator': 0,       # Toroidal component (in lab-frame)
        },
        # Keplerian velocity field
        'Omega_dir': 'cw',
        
        # Image plane 
        'num_alpha': 64,
        'num_beta': 64,
        't_start_obs': 9.25 
    }}

## Flare lightcurves 
---
Polarized image-plane/lightcurves corresponding to ALMA scan times on April 11, 2017. 

In [38]:
flare_path = Path('../data/synthetic_lightcurves/3_gaussians/emission_3d.nc')

locals().update(simulation_params['model'])
emission = xr.load_dataarray(flare_path)

# Random sample pixel ray positions and integrate to avoid numerical artifacts
image_plane = 0
for i in tqdm(range(num_subrays)):
    # Compute geodesics
    geos = bhnerf.kgeo.image_plane_geos(
        spin, np.deg2rad(inclination), 
        num_alpha=num_alpha, num_beta=num_beta, 
        alpha_range=[-fov_M/2, fov_M/2],
        beta_range=[-fov_M/2, fov_M/2],
        randomize_subpixel_rays=True
    )
    t_injection = -float(geos.r_o + fov_M/4)

    # Keplerian velocity field
    rot_sign = {'cw': -1, 'ccw': 1}
    Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
    umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
    g = bhnerf.kgeo.doppler_factor(geos, umu)

    # Magnitude normalized magnetic field in fluid-frame
    b = bhnerf.kgeo.magnetic_field_fluid_frame(geos, umu, **b_consts)
    z_width = 4                                      # maximum disk width [M]
    rmax = fov_M / 2                                 # maximum recovery radius
    rmin = float(bhnerf.constants.isco_pro(spin))    # minimum recovery radius
    domain = np.bitwise_and(np.bitwise_and(np.abs(geos.z) < z_width, geos.r > rmin), geos.r < rmax)
    b_mean = np.sqrt(np.sum(b[domain]**2, axis=-1)).mean()
    b /= b_mean

    # Polarized emission factors (including parallel transport)
    J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)

    image_plane += (1/num_subrays) * bhnerf.emission.image_plane_dynamics(
        emission, geos, Omega, t_frames, t_injection, J, t_start_obs=t_start_obs*units.hr
    )

image_plane = xr.DataArray(
    image_plane, 
    name='Image Plane',
    dims=['t', 'stokes', 'beta', 'alpha'], 
    coords={'t': t_frames, 'stokes': ['I', 'Q', 'U'], 'beta': geos.beta, 'alpha': geos.alpha})
lightcurves = image_plane.sum(['alpha','beta'])

# Normalize image/plane and lightcurves
I, P = lc_average_fluxes(lightcurves) 
emission_scale = I_flux / I
image_plane *= emission_scale
lightcurves *= emission_scale
I, P = lc_average_fluxes(lightcurves) 

# Save image_plane and lightcurves
sim_name = simulation_params['name']
image_path = flare_path.parent.joinpath(sim_name+'_image_plane.nc')
lightcurve_path = flare_path.parent.joinpath(sim_name+'_lightcurve.csv')
image_plane.to_netcdf(image_path)
lightcurves.to_pandas().to_csv(lightcurve_path)

# Update and save simulation parameters
simulation_params['model'].update(emission_scale=emission_scale, P_flux=P)
simulation_params.update(
    flare_path=str(flare_path),
    image_path=str(image_path), 
    lightcurve_path=str(lightcurve_path)
)
with open(flare_path.parent.joinpath('{}_params.yaml'.format(sim_name)), 'w') as file:
    yaml.dump(simulation_params, file, default_flow_style=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [39]:
%matplotlib widget
print('I mean: {}  P mean:{}'.format(I, P))
axes = bhnerf.visualization.plot_stokes_lc(lightcurves, ['I', 'Q', 'U'], t_frames, add_mean=True, plot_qu=True)
plt.savefig(flare_path.parent.joinpath(sim_name + '_lightcurve.pdf'))

fig, axes = plt.subplots(1, 3, figsize=(10, 3))
bhnerf.visualization.animate_movies_synced(
    image_plane.transpose('stokes',...), axes, output=flare_path.parent.joinpath(sim_name + '_image_plane.gif'))

I mean: 0.3  P mean:0.14185841229980198


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Noisy accretion lightcurves 
---
Background accretion is generated as a Gaussian Random Field (GRF) using [pynoisy](https://github.com/aviadlevis/pynoisy)

In [93]:
Q_fracs = [0.25, 0.5, 0.75]
seeds = [0, 1, 2, 3 ,4]

for seed in tqdm(seeds, desc='seeds'):
    for Q_frac in tqdm(Q_fracs, desc='Q_frac', leave=False):
        simulation_params['model'].update(I_flux=2.4, Q_frac=Q_frac)
        simulation_params['name'] = 'sim1_seed{}_Q_{:1.2f}'.format(seed, Q_frac)
        locals().update(simulation_params['model'])

        flare_path = Path('../data/synthetic_lightcurves/accretion_grfs/emission_3d_seed_{}.nc'.format(seed))
        emission = xr.load_dataarray(flare_path)

        # Random sample pixel ray positions and integrate to avoid numerical artifacts
        image_plane = 0
        for i in tqdm(range(num_subrays), leave=False):
            # Compute geodesics
            geos = bhnerf.kgeo.image_plane_geos(
                spin, np.deg2rad(inclination), 
                num_alpha=num_alpha, num_beta=num_beta, 
                alpha_range=[-fov_M/2, fov_M/2],
                beta_range=[-fov_M/2, fov_M/2],
                randomize_subpixel_rays=True
            )
            t_injection = -float(geos.r_o + fov_M/4)

            # Keplerian velocity field
            rot_sign = {'cw': -1, 'ccw': 1}
            Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
            umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
            g = bhnerf.kgeo.doppler_factor(geos, umu)

            # Magnitude normalized magnetic field in fluid-frame
            b = bhnerf.kgeo.magnetic_field_fluid_frame(geos, umu, **b_consts)
            z_width = 4                                      # maximum disk width [M]
            rmax = fov_M / 2                                 # maximum recovery radius
            rmin = float(bhnerf.constants.isco_pro(spin))    # minimum recovery radius
            domain = np.bitwise_and(np.bitwise_and(np.abs(geos.z) < z_width, geos.r > rmin), geos.r < rmax)
            b_mean = np.sqrt(np.sum(b[domain]**2, axis=-1)).mean()
            b /= b_mean

            # Polarized emission factors (including parallel transport)
            J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)

            image_plane += (1/num_subrays) * bhnerf.emission.image_plane_dynamics(emission, geos, Omega, 0.0, 0.0, J, slow_light=False)

        image_plane = xr.DataArray(
            image_plane, 
            name='Image Plane',
            dims=['t', 'stokes', 'beta', 'alpha'], 
            coords={'t': t_frames, 'stokes': ['I', 'Q', 'U'], 'beta': geos.beta, 'alpha': geos.alpha})
        lightcurves = image_plane.sum(['alpha','beta'])

        # Normalize image/plane and lightcurves
        I, P = lc_average_fluxes(lightcurves) 
        emission_scale = I_flux / I
        image_plane *= emission_scale
        lightcurves *= emission_scale
        I, P_mean = lc_average_fluxes(lightcurves)
        P_std = float(np.std(lightcurves[:,1] + 1j*lightcurves[:,2]))

        # Save image_plane and lightcurves
        sim_name = simulation_params['name']
        image_path = flare_path.parent.joinpath(sim_name+'_image_plane.nc')
        lightcurve_path = flare_path.parent.joinpath(sim_name+'_lightcurve.csv')
        image_plane.to_netcdf(image_path)
        lightcurves.to_pandas().to_csv(lightcurve_path)

        # Update and save simulation parameters
        simulation_params['model'].update(emission_scale=emission_scale, P_flux=P_mean, P_std=P_std)
        simulation_params.update(
            flare_path=str(flare_path),
            image_path=str(image_path), 
            lightcurve_path=str(lightcurve_path)
        )
        with open(flare_path.parent.joinpath('{}_params.yaml'.format(sim_name)), 'w') as file:
            yaml.dump(simulation_params, file, default_flow_style=False)

        %matplotlib widget
        print('I mean: {}  P mean:{}  P std:{}'.format(I, P_mean, P_std))
        axes = bhnerf.visualization.plot_stokes_lc(lightcurves, ['I', 'Q', 'U'], t_frames, add_mean=True, plot_qu=True)
        plt.savefig(flare_path.parent.joinpath(sim_name + '_lightcurve.pdf'))

seeds:   0%|          | 0/5 [00:00<?, ?it/s]

Q_frac:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

I mean: 2.400000000000001  P mean:0.12156061457053331  P std:0.013395542032903319


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

I mean: 2.399999999999999  P mean:0.24885242732902177  P std:0.03430364017335448


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/10 [00:00<?, ?it/s]

I mean: 2.3999999999999995  P mean:0.38139732791585984  P std:0.050225172036325316


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
%matplotlib widget
dynamic_range = 1.e3
gamma = 0.1
gamma_fn = lambda movie : (movie + np.max(movie) / dynamic_range)**(gamma)
log_fn = lambda movie: np.log10(movie + np.max(movie) / dynamic_range)
log_fn(image_plane.sel(stokes='I')).visualization.animate()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …